In [1]:
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds

epochs = 30
batch_size = 64

!rm -rf ./logs/hypothyroid/*

In [2]:
import numpy as np
from sklearn import preprocessing
from sklearn.datasets import fetch_openml


openml_dataset = fetch_openml(
    name="hypothyroid", version=2, as_frame=False, parser="liac-arff"
)

n_total, dims = openml_dataset.data.shape
n = int(n_total * 0.7)

X = openml_dataset.data.astype(np.float32)
y = preprocessing.LabelEncoder().fit_transform(openml_dataset.target)

ds_raw = tf.data.Dataset.from_tensor_slices((X, y)).shuffle(n_total)

ds_train_raw = ds_raw.take(n)
ds_test_raw = ds_raw.skip(n)

n_classes = np.unique(openml_dataset.target).shape[0]

print("n: ", n, "n_classes: ", n_classes, "dims: ", dims)


n:  2640 n_classes:  2 dims:  29


In [3]:
ds_train_normalized = ds_train_raw.cache()

ds_test_normalized = ds_test_raw.cache()


def prepare(ds, batch_size=batch_size):
    return ds.shuffle(n).batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [4]:
RandomFourierFeatures = keras.layers.experimental.RandomFourierFeatures

model_svm = keras.Sequential(
    [
        layers.Input(shape=(dims,)),
        RandomFourierFeatures(
            output_dim=2000, scale=10.0, kernel_initializer="gaussian"
        ),
        layers.Dense(units=n_classes),
    ]
)
model_svm.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.hinge,
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
)

modeldir = "./logs/hypothyroid/linear-8192-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_svm.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(
            log_dir=modeldir + "/log",
            histogram_freq=1,
        )
    ],
)
model_svm.save(modeldir + "/model")


Epoch 1/30
42/42 [==============================] - 2s 11ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 2/30
42/42 [==============================] - 0s 8ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 3/30
42/42 [==============================] - 0s 6ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 4/30
42/42 [==============================] - 0s 7ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 5/30
42/42 [==============================] - 0s 7ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 6/30
42/42 [==============================] - 0s 7ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 7/30
42/42 [==============================] - 0s 6ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 8/30
42/42 [==============================] - 0s 7ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 9

In [5]:
def gen_samples(psi, t=1000):
    return [
        list(
            ds_train_raw.shuffle(n)
            .take(psi)
            .batch(psi)
            .as_numpy_iterator()
        )[0][0]
        for _ in range(t)
    ]


def _tf_ann(X, samples, p=2, soft=True):
    m_dis = None
    for i in range(samples.shape[0]):
        i_sample = samples[i : i + 1, :]
        l_dis = tf.math.reduce_sum((X - i_sample) ** p, axis=1, keepdims=True) ** (
            1 / p
        )
        if m_dis is None:
            m_dis = l_dis
        else:
            m_dis = tf.concat([m_dis, l_dis], 1)

    if soft:
        feature_map = tf.nn.softmax(-m_dis, axis=0)
    else:
        feature_map = tf.one_hot(tf.math.argmax(-m_dis, axis=1), samples.shape[0])
    # l_dis_min = tf.math.reduce_sum(m_dis * feature_map, axis=0)
    return feature_map


In [6]:
class IsolationEncodingLayer(layers.Layer):
    def __init__(self, samples, p=2, soft=True, **kwargs):
        super(IsolationEncodingLayer, self).__init__(**kwargs)
        self.samples = samples
        self.p = p
        self.soft = soft

    def call(self, inputs):
        return _tf_ann(inputs, self.samples, self.p, self.soft)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "samples": self.samples,
                "p": self.p,
                "soft": self.soft,
            }
        )
        return config


def build_model(t_samples, p=2, soft=True):
    t = len(t_samples)
    if t <= 0:
        raise ValueError("t <= 0")
    _, dims = t_samples[0].shape

    inputs = keras.Input(name="inputs_x", shape=(dims,))
    lambdas = [
        IsolationEncodingLayer(t_samples[i], p=p, soft=soft, name="ann_{}".format(i))(
            inputs
        )
        for i in range(t)
    ]
    concatenated = layers.Concatenate(axis=1, name="concatenated")(lambdas)
    outputs = layers.Dense(units=n_classes, name="outputs_y")(concatenated)

    model = keras.Model(name="isolation_encoding", inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.hinge,
        metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    )
    return model


In [7]:
def _tf_ann_weighted(X, samples, sample_weights, p=2, soft=True):
    m_dis = None  # [n, psi]
    for i in range(samples.shape[0]):
        i_sample = samples[i : i + 1, :]  # [i, dims]
        l_dis = tf.math.reduce_sum((X - i_sample) ** p, axis=1, keepdims=True) ** (
            1 / p
        )  # [n, 1]
        if m_dis is None:
            m_dis = l_dis
        else:
            m_dis = tf.concat([m_dis, l_dis], 1)

    m_dis = m_dis * sample_weights

    if soft:
        feature_map = tf.nn.softmax(-m_dis, axis=0)
    else:
        feature_map = tf.one_hot(tf.math.argmax(-m_dis, axis=1), samples.shape[0])
    # l_dis_min = tf.math.reduce_sum(m_dis * feature_map, axis=0)
    return feature_map


class FlexibleIsolationEncodingLayer(layers.Layer):
    def __init__(self, samples, p=2, **kwargs):
        super(FlexibleIsolationEncodingLayer, self).__init__(**kwargs)
        self.samples = samples
        self.p = p

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.sample_weights = self.add_weight(
            name="dimential_weights",
            shape=(
                1,
                self.samples.shape[0],
            ),
            initializer="ones",
            trainable=True,
        )
        super(FlexibleIsolationEncodingLayer, self).build(input_shape)

    def call(self, inputs):
        return _tf_ann_weighted(inputs, self.samples, self.sample_weights, self.p)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "samples": self.samples,
                "p": self.p,
            }
        )
        return config


def build_flex_model(t_samples, p=2):
    t = len(t_samples)
    if t <= 0:
        raise ValueError("t <= 0")
    _, dims = t_samples[0].shape

    inputs = keras.Input(name="inputs_x", shape=(dims,))
    lambdas = [
        FlexibleIsolationEncodingLayer(t_samples[i], p=p, name="ann_flex_{}".format(i))(
            inputs
        )
        for i in range(t)
    ]
    concatenated = layers.Concatenate(axis=1, name="concatenated")(lambdas)
    outputs = layers.Dense(units=n_classes, name="outputs_y")(concatenated)

    model = keras.Model(name="isolation_encoding", inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.hinge,
        metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    )
    return model


In [8]:
t_samples = gen_samples(psi=20, t=50)


model_hard_20_50 = build_model(t_samples, soft=False)
modeldir = "./logs/hypothyroid/hard-20x50-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_hard_20_50.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_hard_20_50.save(modeldir + "/model")


Epoch 1/30
42/42 [==============================] - 27s 334ms/step - loss: 0.3575 - acc: 0.8242 - val_loss: 0.1600 - val_acc: 0.0000e+00
Epoch 2/30
42/42 [==============================] - 6s 144ms/step - loss: 0.1717 - acc: 0.0000e+00 - val_loss: 0.1460 - val_acc: 0.0000e+00
Epoch 3/30
42/42 [==============================] - 6s 146ms/step - loss: 0.1590 - acc: 0.2000 - val_loss: 0.1391 - val_acc: 1.0000
Epoch 4/30
42/42 [==============================] - 6s 150ms/step - loss: 0.1569 - acc: 0.2667 - val_loss: 0.1406 - val_acc: 0.0000e+00
Epoch 5/30
42/42 [==============================] - 6s 145ms/step - loss: 0.1560 - acc: 0.5152 - val_loss: 0.1393 - val_acc: 1.0000
Epoch 6/30
42/42 [==============================] - 6s 147ms/step - loss: 0.1562 - acc: 0.6606 - val_loss: 0.1390 - val_acc: 1.0000
Epoch 7/30
42/42 [==============================] - 6s 145ms/step - loss: 0.1560 - acc: 0.6121 - val_loss: 0.1391 - val_acc: 0.0000e+00
Epoch 8/30
42/42 [==============================] - 6s 

In [9]:
model_soft_20_50 = build_model(t_samples, soft=True)
modeldir = "./logs/hypothyroid/soft-20x50-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_soft_20_50.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_soft_20_50.save(modeldir + "/model")


Epoch 1/30
42/42 [==============================] - 31s 362ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 2/30
42/42 [==============================] - 7s 167ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 3/30
42/42 [==============================] - 7s 170ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 4/30
42/42 [==============================] - 7s 161ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 5/30
42/42 [==============================] - 7s 161ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 6/30
42/42 [==============================] - 7s 160ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 7/30
42/42 [==============================] - 7s 162ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 8/30
42/42 [==============================] - 7s 163ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc

In [10]:
model_flex_20_50 = build_flex_model(t_samples)
# model_flex_20_50.summary()
modeldir = "./logs/hypothyroid/flex-20x50-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_flex_20_50.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_flex_20_50.save(modeldir + "/model")


Epoch 1/30
42/42 [==============================] - 33s 396ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 2/30
42/42 [==============================] - 8s 188ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 3/30
42/42 [==============================] - 8s 186ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 4/30
42/42 [==============================] - 8s 182ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 5/30
42/42 [==============================] - 8s 185ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 6/30
42/42 [==============================] - 8s 182ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 7/30
42/42 [==============================] - 7s 178ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 8/30
42/42 [==============================] - 7s 177ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc

In [11]:
t_samples = gen_samples(psi=100, t=10)

model_hard_100_10 = build_model(t_samples, soft=False)
modeldir = "./logs/hypothyroid/hard-100x10-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_hard_100_10.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_hard_100_10.save(modeldir + "/model")


Epoch 1/30
42/42 [==============================] - 25s 310ms/step - loss: 0.7367 - acc: 1.0000 - val_loss: 0.5239 - val_acc: 1.0000
Epoch 2/30
42/42 [==============================] - 6s 140ms/step - loss: 0.3483 - acc: 1.0000 - val_loss: 0.1552 - val_acc: 1.0000
Epoch 3/30
42/42 [==============================] - 6s 141ms/step - loss: 0.1605 - acc: 0.5091 - val_loss: 0.1426 - val_acc: 0.0000e+00
Epoch 4/30
42/42 [==============================] - 6s 140ms/step - loss: 0.1579 - acc: 0.0000e+00 - val_loss: 0.1387 - val_acc: 0.0000e+00
Epoch 5/30
42/42 [==============================] - 6s 140ms/step - loss: 0.1550 - acc: 0.2909 - val_loss: 0.1381 - val_acc: 0.0000e+00
Epoch 6/30
42/42 [==============================] - 6s 142ms/step - loss: 0.1549 - acc: 0.0970 - val_loss: 0.1380 - val_acc: 1.0000
Epoch 7/30
42/42 [==============================] - 6s 140ms/step - loss: 0.1547 - acc: 0.4606 - val_loss: 0.1381 - val_acc: 0.0000e+00
Epoch 8/30
42/42 [==============================] - 6s 

In [12]:
model_soft_100_10 = build_model(t_samples, soft=True)
modeldir = "./logs/hypothyroid/soft-100x10-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_soft_100_10.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_soft_100_10.save(modeldir + "/model")


Epoch 1/30
42/42 [==============================] - 27s 329ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 2/30
42/42 [==============================] - 6s 152ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 3/30
42/42 [==============================] - 6s 152ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 4/30
42/42 [==============================] - 6s 149ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 5/30
42/42 [==============================] - 6s 150ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 6/30
42/42 [==============================] - 6s 149ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 7/30
42/42 [==============================] - 6s 149ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 8/30
42/42 [==============================] - 6s 147ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc

In [13]:
model_flex_100_10 = build_flex_model(t_samples)
# model_flex_20_50.summary()
modeldir = "./logs/hypothyroid/flex-100x10-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_flex_100_10.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_flex_100_10.save(modeldir + "/model")


Epoch 1/30
42/42 [==============================] - 26s 310ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 2/30
42/42 [==============================] - 6s 144ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 3/30
42/42 [==============================] - 6s 145ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 4/30
42/42 [==============================] - 6s 143ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 5/30
42/42 [==============================] - 6s 137ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 6/30
42/42 [==============================] - 6s 145ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 7/30
42/42 [==============================] - 6s 139ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc: 1.0000
Epoch 8/30
42/42 [==============================] - 6s 141ms/step - loss: nan - acc: 1.0000 - val_loss: nan - val_acc